In [1]:
import os
import numpy as np
import cv2
from glob import glob
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler

In [2]:
_IMAGE_SIZE = 256

In [16]:
def cal_for_frames(video_path):
    print(video_path)
    frames = glob(os.path.join(video_path, '*.png'))
    frames.sort()

    flow = []
    prev = cv2.imread(frames[0])
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    for i, frame_curr in enumerate(frames):
        curr = cv2.imread(frame_curr)
        curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
        tmp_flow = compute_TVL1(prev, curr)
        flow.append(tmp_flow)
        prev = curr

    return flow


In [22]:

def compute_TVL1(prev, curr, bound=15):
    """Compute the TV-L1 optical flow."""
    TVL1 = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = TVL1.calc(prev, curr, None)
    #flow_frame = np.clip(flow_frame, -20, 20)
    assert flow.dtype == np.float32

    flow = (flow + bound) * (255.0 / (2*bound))
    flow = np.round(flow).astype(int)
    flow[flow >= 255] = 255
    flow[flow <= 0] = 0

    return flow

In [23]:
def save_flow(video_flows, flow_path):
    for i, flow in enumerate(video_flows):
        cv2.imwrite(os.path.join(flow_path.format('u'), "{:06d}.jpg".format(i)),
                    flow[:, :, 0])
        cv2.imwrite(os.path.join(flow_path.format('v'), "{:06d}.jpg".format(i)),
                    flow[:, :, 1])


In [24]:
def extract_flow(args):
    video_path, flow_path = args
    flow = cal_for_frames(video_path)
    save_flow(flow, flow_path)
    print('complete:' + flow_path)
    return

In [25]:
frame_path='/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/cricket/u/' #This is the path where  images sampled from the video are saved
flow_path='/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/cricket/v' # this is the path where you want save the flow files.

In [26]:
# video_path='train/u/' 
flow = cal_for_frames(frame_path)

/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/cricket/u/


In [28]:
save_flow(flow, flow_path)

# Convert images to .npyfile as descibed in  https://github.com/deepmind/kinetics-i3d

In [29]:
# scaler = MinMaxScaler(feature_range=(-1,1))
# print(scaler.transform(data))

In [35]:
def norm_rgb(rgb_path,nchannel):
    #print(video_path)
    npy_file=[]
    frames = glob(os.path.join(rgb_path, '*.png'))
    frames.sort()
    print(len(frames))
    for frame in frames:
        img=cv2.imread(os.path.join(frame_path,frame))
        img_new=(cv2.resize(img,(224,224))).astype(float)
        img_norm=np.divide(2*(img_new-img_new.min()),(img_new.max()-img_new.min()))-1
        
        npy_file.append(img_norm)
        
    npy_file=np.reshape(np.asarray(npy_file),(1,len(frames),224,224,nchannel))
    np.save('data_input_rgb_cricket.npy',npy_file)
    return npy_file
        

In [39]:
def norm_flow(flow_path,nchannel):
    #print(video_path)
    npy_file=[]
    frames = glob(os.path.join(flow_path, '*.jpg'))
    frames.sort()
    print(len(frames))
    for frame in frames:
        img=cv2.imread(os.path.join(frame_path,frame))
        #img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_new=(cv2.resize(img,(224,224))).astype(float)
        img_norm=np.divide(2*(img_new-img_new.min()),(img_new.max()-img_new.min()))-1
        
        npy_file.append(img_norm[:,:,:-1])
        
    npy_file=np.reshape(np.asarray(npy_file),(1,len(frames),224,224,nchannel-1))
    np.save('data_input_flow_cricket.npy',npy_file)
    return npy_file

In [40]:
np_file=norm_rgb(frame_path,3)

79


In [41]:
np_file=norm_flow(flow_path,3)

79


/home/lenovo/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [76]:
np.asarray(np_file).shape

(301, 224, 224, 2)